Primero vemos el caso homogéneo para nuestro modelo a modo de validación.

In [ ]:
using Pkg
Pkg.add("CoordinateTransformations")
Pkg.add("StaticArrays")

# Validación McBride

Usamos un modelo 2D (x,z) con $c(z)=332+0.6z$. Ponemos un nodo cada 1m. Comparamos con la gráfica 3.9 de McBride 2017. La fuente se situa en (0,100) y los receptores aproximadamente en (58,280)m,(150,220)m y (86,117)m . dq/dt=dq/dz*dz/dt=dq/dz*1/qz

In [1]:
#Planteamos nuestro método con estas ODE
using DifferentialEquations
T=20.
#Definimos la ODE 
function q!(dq, q, p, z)
    q_aux=sqrt(q[1]^2 + q[2]^2 )
    dq[1]=0
    dq[2] = -q_aux*q[2]*0.6
end
c_air=332+100*0.6
# Definimos las condiciones iniciales y los parámetros

T_z_0m = T

# Definimps el rango de valores de z. Debemos tener cuidado al optimizar, dado
# que si algún valor (coordenada z) optimizado está fuera de zspan no podemos evaluarlo.
zspan = (100., 500.)

# Definimos los parámetros
p = [T_z_0m]

function Hamiltonian_l(qx::Float64,qz_z_linear, SegmentsCoords::Array{Float64, 2})
    #Calculamos los traveltime entre cada segmento tomando la media de las q entre cada segmento (aproximación regla Trapecio)
    taus=zeros(length(SegmentsCoords[:,1])-1)
    for i in 1:(length(SegmentsCoords[:,1])-1)
        qz_m=(qz_z_linear(SegmentsCoords[i,2])+qz_z_linear(SegmentsCoords[i+1,2]))/2
        q_m=sqrt(qx^2+qz_m^2)
        aux1=(SegmentsCoords[i+1,1]-SegmentsCoords[i,1])
        aux2=(SegmentsCoords[i+1,2]-SegmentsCoords[i,2])
        taus[i] = q_m*sqrt(aux1^2+aux2^2)
        #aux_1=[qx*SegmentsCoords[i,1],qz_m*SegmentsCoords[i,2]]
        #aux_2=[qx*SegmentsCoords[i+1,1],qz_m*SegmentsCoords[i+1,2]]
        #taus[i]=abs(norm(aux_2)-norm(aux_1))
    end
    #Sumamos el travel time de cada segmento para obtener el total
    total_travel_time = sum(taus)
    return total_travel_time
end

Hamiltonian_l (generic function with 1 method)

In [38]:
using LinearAlgebra
function Slowness_Spherical_2D(Point1::Array{Float64, 1}, Point2::Array{Float64, 1}, c_source)
    q0=1/c_source
    x1,z1=Point2-Point1
    r=norm(Point2-Point1)
    theta=rad2deg(acos(z1/r)) 
    qx=abs(q0*sind(theta))
    qz=abs(q0*cosd(theta))
    return qx,qz,theta
end
Slowness_Spherical_2D([0.,100.],[60.,292.74], 332)
    
    

(0.0008952745668717472, 0.0028759203336476765, 17.2913756044773)

In [39]:
using Interpolations
using Optim
using Statistics
@time begin  
Source=[0.,100.]
Receiver=[60.,292.74]
#Con la fuente y el receptor damos un ángulo inicial.

q_x,q_z,theta_i=Slowness_Spherical_2D(Source,Receiver, c_air)
println("Ángulo 1 segmento: ", theta_i)
u0 = [q_x, q_z]
prob = ODEProblem(q!, u0, zspan, p)
sol = solve(prob, RK4(),dt=1e-5,saveat=Source[2]:1:500.)

#Guardamos los valores de vec(q)
qx_vals=zeros(length(sol.u))
qz_vals=zeros(length(sol.u))
for i in 1:length(sol.u)
    qx_vals[i]=sol.u[i][1]
    qz_vals[i]=sol.u[i][2]
end
###### 
qx=sum(qx_vals)/length(qx_vals)
######
z_values_linear = collect(range(Source[2], stop=500., length=length(qz_vals)))
qz_z_linear=LinearInterpolation(z_values_linear, qz_vals)

#Optimizamos las coordenadas de las uniones entre segmentos
function objective(SegmentsCoords)
    # Primero nos aseguramos de que SourcePoint y ReceiverPoint no se optimizan pero que se incluyen en SegmentsCoords
    FullSegmentsCoords = vcat(reshape(Source, 1, :), SegmentsCoords, reshape(Receiver, 1, :))
    last_angle_traveltime = Hamiltonian_l(qx, qz_z_linear, FullSegmentsCoords)
    # Calculamos las distancias entre segmentos consecutivos
    segment_distances = diff(FullSegmentsCoords, dims=1)
    # Calculamos la desviación estándar de las distancias
    distance_std_normalized = Statistics.std(segment_distances)/(Receiver[2]-Source[2])
    # Agregamos una penalización a la función objetivo basada en la desviación estándar de las distancias
    return last_angle_traveltime 
end

n=8 #Numero de iteraciones
#Definimos una matriz de matrices Plot_Coords del tamaño del bucle o mayor para luego graficar la distancia de los puntos a una recta
Plot_Coords = Matrix{Matrix{Float64}}(undef, n, 1)
#Iteramos la condición inicial
initial_SegmentsCoords = zeros(1, 2); #c.i.
    
for i in 1:n
    if i == 1
        initial_SegmentsCoords[1, :] = Source + (Receiver - Source) / 2
    end
    lower_bounds = fill(-100., size(initial_SegmentsCoords))
    upper_bounds = fill(600., size(initial_SegmentsCoords))
    inner_optimizer = LBFGS()
    optimizer_with_constraints = Fminbox(inner_optimizer)
    result = optimize(objective, lower_bounds, upper_bounds, initial_SegmentsCoords, optimizer_with_constraints)
    Optimal_SegmentsCoords = result.minimizer;
    Full_Optimal_SegmentsCoords= vcat(reshape(Source, 1, :), Optimal_SegmentsCoords, reshape(Receiver, 1, :))
    #Almacenamos las coordenadas de la optimización incluyendo la fuente y el receptor para luego graficar
    Plot_Coords[i]=Full_Optimal_SegmentsCoords
    #Evaluamos el travel time para los distintos números de segmentos planteados
    total_time = Hamiltonian_l(qx, qz_z_linear, Full_Optimal_SegmentsCoords)
    println("Num segments: ", 2^(i-1), ". Traveltime:", total_time, " s")
    println("Iteraciones realizadas: ", result.iterations, ". Evaluaciones de la función: ", result.f_calls)
        

    copy_Optimal_SegmentsCoords = zeros(2 * size(Optimal_SegmentsCoords, 1) + 1, 2)
    # Calculamos copy_optimal_SegmentsCoords 
    for j in 1:2 * size(Optimal_SegmentsCoords, 1) + 1
        if j==1
            copy_Optimal_SegmentsCoords[j,:]=(Source+Optimal_SegmentsCoords[1,:])/2
        elseif j==2*size(Optimal_SegmentsCoords,1)+1
            copy_Optimal_SegmentsCoords[j,:]=(Optimal_SegmentsCoords[end,:]+Receiver)/2
        else
            if iseven(j)
                copy_Optimal_SegmentsCoords[j,:]=Optimal_SegmentsCoords[Int(j/2),:]
            else
                copy_Optimal_SegmentsCoords[j,:]=(Optimal_SegmentsCoords[Int((j-1)/2),:]+Optimal_SegmentsCoords[Int((j+1)/2),:])/2
            end
        end
    end
    #Aquí iteramos la condición inicial
    initial_SegmentsCoords = copy(copy_Optimal_SegmentsCoords)
        
    q_x,q_z,theta_i=Slowness_Spherical_2D(Source,Optimal_SegmentsCoords[1,:], c_air)
    println("Ángulo ", 2^i, "segmentos: ", theta_i)
    u0 = [q_x, q_z]
    prob = ODEProblem(q!, u0, zspan, p)
    sol = solve(prob, RK4(),dt=1e-5,saveat=Source[2]:1:500.)

    #Guardamos los valores de vec(q)
    qx_vals=zeros(length(sol.u))
    qz_vals=zeros(length(sol.u))
    for i in 1:length(sol.u)
        qx_vals[i]=sol.u[i][1]
        qz_vals[i]=sol.u[i][2]
    end
    ###### 
    qx=sum(qx_vals)/length(qx_vals)
    ######
    z_values_linear = collect(range(Source[2], stop=500., length=length(qz_vals)))
    qz_z_linear=LinearInterpolation(z_values_linear, qz_vals)
end
traveltime=Hamiltonian_l(qx, qz_z_linear, Plot_Coords[n])
println("Num segments: ", 2^n, ". Traveltime:", traveltime, " s")
P1_ImPropia=deepcopy(Plot_Coords[n]);
""
end

Ángulo 1 segmento: 17.2913756044773
Num segments: 1. Traveltime:0.4579307921598585 s
Iteraciones realizadas: 2. Evaluaciones de la función: 68
Ángulo 2segmentos: 16.186963926984895
Num segments: 2. Traveltime:0.4564361826379604 s
Iteraciones realizadas: 2. Evaluaciones de la función: 331
Ángulo 4segmentos: 15.66888437151421
Num segments: 4. Traveltime:0.45592104489951535 s
Iteraciones realizadas: 2. Evaluaciones de la función: 1094
Ángulo 8segmentos: 15.417558863055273
Num segments: 8. Traveltime:0.45572251126734836 s
Iteraciones realizadas: 2. Evaluaciones de la función: 3516
Ángulo 16segmentos: 15.291290116899114
Num segments: 16. Traveltime:0.4556367369061331 s
Iteraciones realizadas: 2. Evaluaciones de la función: 3241
Ángulo 32segmentos: 15.228652624142322
Num segments: 32. Traveltime:0.45559758157235836 s
Iteraciones realizadas: 4. Evaluaciones de la función: 11813
Ángulo 64segmentos: 15.199036551513625
Num segments: 64. Traveltime:0.4555798993325693 s
Iteraciones realizadas: 6. 

""

In [40]:
using Interpolations
using Optim
using Statistics
@time begin  
Source=[0.,100.]
Receiver=[150,221.49]
#Con la fuente y el receptor damos un ángulo inicial.

q_x,q_z,theta_i=Slowness_Spherical_2D(Source,Receiver, c_air)
println("Ángulo 1 segmento: ", theta_i)
u0 = [q_x, q_z]
prob = ODEProblem(q!, u0, zspan, p)
sol = solve(prob, RK4(),dt=1e-5,saveat=Source[2]:1:500.)

#Guardamos los valores de vec(q)
qx_vals=zeros(length(sol.u))
qz_vals=zeros(length(sol.u))
for i in 1:length(sol.u)
    qx_vals[i]=sol.u[i][1]
    qz_vals[i]=sol.u[i][2]
end
###### 
qx=sum(qx_vals)/length(qx_vals)
######
z_values_linear = collect(range(Source[2], stop=500., length=length(qz_vals)))
qz_z_linear=LinearInterpolation(z_values_linear, qz_vals)

#Optimizamos las coordenadas de las uniones entre segmentos
function objective(SegmentsCoords)
    # Primero nos aseguramos de que SourcePoint y ReceiverPoint no se optimizan pero que se incluyen en SegmentsCoords
    FullSegmentsCoords = vcat(reshape(Source, 1, :), SegmentsCoords, reshape(Receiver, 1, :))
    last_angle_traveltime = Hamiltonian_l(qx, qz_z_linear, FullSegmentsCoords)
    # Calculamos las distancias entre segmentos consecutivos
    segment_distances = diff(FullSegmentsCoords, dims=1)
    # Calculamos la desviación estándar de las distancias
    distance_std_normalized = Statistics.std(segment_distances)/(Receiver[2]-Source[2])
    # Agregamos una penalización a la función objetivo basada en la desviación estándar de las distancias
    return last_angle_traveltime 
end

n=8 #Numero de iteraciones
#Definimos una matriz de matrices Plot_Coords del tamaño del bucle o mayor para luego graficar la distancia de los puntos a una recta
Plot_Coords = Matrix{Matrix{Float64}}(undef, n, 1)
#Iteramos la condición inicial
initial_SegmentsCoords = zeros(1, 2); #c.i.
    
for i in 1:n
    if i == 1
        initial_SegmentsCoords[1, :] = Source + (Receiver - Source) / 2
    end
    lower_bounds = fill(-100., size(initial_SegmentsCoords))
    upper_bounds = fill(600., size(initial_SegmentsCoords))
    inner_optimizer = LBFGS()
    optimizer_with_constraints = Fminbox(inner_optimizer)
    result = optimize(objective, lower_bounds, upper_bounds, initial_SegmentsCoords, optimizer_with_constraints)
    Optimal_SegmentsCoords = result.minimizer;
    Full_Optimal_SegmentsCoords= vcat(reshape(Source, 1, :), Optimal_SegmentsCoords, reshape(Receiver, 1, :))
    #Almacenamos las coordenadas de la optimización incluyendo la fuente y el receptor para luego graficar
    Plot_Coords[i]=Full_Optimal_SegmentsCoords
    #Evaluamos el travel time para los distintos números de segmentos planteados
    total_time = Hamiltonian_l(qx, qz_z_linear, Full_Optimal_SegmentsCoords)
    println("Num segments: ", 2^(i-1), ". Traveltime:", total_time, " s")
    println("Iteraciones realizadas: ", result.iterations, ". Evaluaciones de la función: ", result.f_calls)
        

    copy_Optimal_SegmentsCoords = zeros(2 * size(Optimal_SegmentsCoords, 1) + 1, 2)
    # Calculamos copy_optimal_SegmentsCoords 
    for j in 1:2 * size(Optimal_SegmentsCoords, 1) + 1
        if j==1
            copy_Optimal_SegmentsCoords[j,:]=(Source+Optimal_SegmentsCoords[1,:])/2
        elseif j==2*size(Optimal_SegmentsCoords,1)+1
            copy_Optimal_SegmentsCoords[j,:]=(Optimal_SegmentsCoords[end,:]+Receiver)/2
        else
            if iseven(j)
                copy_Optimal_SegmentsCoords[j,:]=Optimal_SegmentsCoords[Int(j/2),:]
            else
                copy_Optimal_SegmentsCoords[j,:]=(Optimal_SegmentsCoords[Int((j-1)/2),:]+Optimal_SegmentsCoords[Int((j+1)/2),:])/2
            end
        end
    end
    #Aquí iteramos la condición inicial
    initial_SegmentsCoords = copy(copy_Optimal_SegmentsCoords)
        
    q_x,q_z,theta_i=Slowness_Spherical_2D(Source,Optimal_SegmentsCoords[1,:], c_air)
    println("Ángulo ", 2^i, "segmentos: ", theta_i)
    u0 = [q_x, q_z]
    prob = ODEProblem(q!, u0, zspan, p)
    sol = solve(prob, RK4(),dt=1e-5,saveat=Source[2]:1:500.)

    #Guardamos los valores de vec(q)
    qx_vals=zeros(length(sol.u))
    qz_vals=zeros(length(sol.u))
    for i in 1:length(sol.u)
        qx_vals[i]=sol.u[i][1]
        qz_vals[i]=sol.u[i][2]
    end
    ###### 
    qx=sum(qx_vals)/length(qx_vals)
    ######
    z_values_linear = collect(range(Source[2], stop=500., length=length(qz_vals)))
    qz_z_linear=LinearInterpolation(z_values_linear, qz_vals)
end
traveltime=Hamiltonian_l(qx, qz_z_linear, Plot_Coords[n])
println("Num segments: ", 2^n, ". Traveltime:", traveltime, " s")
P2_ImPropia=deepcopy(Plot_Coords[n]);
""
end

Ángulo 1 segmento: 50.9948339771856
Num segments: 1. Traveltime:0.4763302707837451 s
Iteraciones realizadas: 2. Evaluaciones de la función: 86
Ángulo 2segmentos: 49.80986593658793
Num segments: 2. Traveltime:0.47538155200668897 s
Iteraciones realizadas: 2. Evaluaciones de la función: 236
Ángulo 4segmentos: 49.13685626130963
Num segments: 4. Traveltime:0.4748711912280873 s
Iteraciones realizadas: 2. Evaluaciones de la función: 2879
Ángulo 8segmentos: 48.78896077994655
Num segments: 8. Traveltime:0.4746141350296325 s
Iteraciones realizadas: 2. Evaluaciones de la función: 6592
Ángulo 16segmentos: 48.62627312266357
Num segments: 16. Traveltime:0.4744953385340171 s
Iteraciones realizadas: 5. Evaluaciones de la función: 16758
Ángulo 32segmentos: 48.52739199150674
Num segments: 32. Traveltime:0.47442378894023046 s
Iteraciones realizadas: 4. Evaluaciones de la función: 10405
Ángulo 64segmentos: 48.464512919753446
Num segments: 64. Traveltime:0.4743784759429785 s
Iteraciones realizadas: 2. Eval

""

In [41]:
using Interpolations
using Optim
using Statistics
@time begin  
Source=[0.,100.]
Receiver=[86,117.16]
#Con la fuente y el receptor damos un ángulo inicial.

q_x,q_z,theta_i=Slowness_Spherical_2D(Source,Receiver, c_air)
println("Ángulo 1 segmento: ", theta_i)
u0 = [q_x, q_z]
prob = ODEProblem(q!, u0, zspan, p)
sol = solve(prob, RK4(),dt=1e-5,saveat=Source[2]:1:500.)

#Guardamos los valores de vec(q)
qx_vals=zeros(length(sol.u))
qz_vals=zeros(length(sol.u))
for i in 1:length(sol.u)
    qx_vals[i]=sol.u[i][1]
    qz_vals[i]=sol.u[i][2]
end
###### 
qx=sum(qx_vals)/length(qx_vals)
######
z_values_linear = collect(range(Source[2], stop=500., length=length(qz_vals)))
qz_z_linear=LinearInterpolation(z_values_linear, qz_vals)

#Optimizamos las coordenadas de las uniones entre segmentos
function objective(SegmentsCoords)
    # Primero nos aseguramos de que SourcePoint y ReceiverPoint no se optimizan pero que se incluyen en SegmentsCoords
    FullSegmentsCoords = vcat(reshape(Source, 1, :), SegmentsCoords, reshape(Receiver, 1, :))
    last_angle_traveltime = Hamiltonian_l(qx, qz_z_linear, FullSegmentsCoords)
    # Calculamos las distancias entre segmentos consecutivos
    segment_distances = diff(FullSegmentsCoords, dims=1)
    # Calculamos la desviación estándar de las distancias
    distance_std_normalized = Statistics.std(segment_distances)/(Receiver[2]-Source[2])
    # Agregamos una penalización a la función objetivo basada en la desviación estándar de las distancias
    return last_angle_traveltime 
end

n=8 #Numero de iteraciones
#Definimos una matriz de matrices Plot_Coords del tamaño del bucle o mayor para luego graficar la distancia de los puntos a una recta
Plot_Coords = Matrix{Matrix{Float64}}(undef, n, 1)
#Iteramos la condición inicial
initial_SegmentsCoords = zeros(1, 2); #c.i.
    
for i in 1:n
    if i == 1
        initial_SegmentsCoords[1, :] = Source + (Receiver - Source) / 2
    end
    lower_bounds = fill(-100., size(initial_SegmentsCoords))
    upper_bounds = fill(600., size(initial_SegmentsCoords))
    inner_optimizer = LBFGS()
    optimizer_with_constraints = Fminbox(inner_optimizer)
    result = optimize(objective, lower_bounds, upper_bounds, initial_SegmentsCoords, optimizer_with_constraints)
    Optimal_SegmentsCoords = result.minimizer;
    Full_Optimal_SegmentsCoords= vcat(reshape(Source, 1, :), Optimal_SegmentsCoords, reshape(Receiver, 1, :))
    #Almacenamos las coordenadas de la optimización incluyendo la fuente y el receptor para luego graficar
    Plot_Coords[i]=Full_Optimal_SegmentsCoords
    #Evaluamos el travel time para los distintos números de segmentos planteados
    total_time = Hamiltonian_l(qx, qz_z_linear, Full_Optimal_SegmentsCoords)
    println("Num segments: ", 2^(i-1), ". Traveltime:", total_time, " s")
    println("Iteraciones realizadas: ", result.iterations, ". Evaluaciones de la función: ", result.f_calls)
        

    copy_Optimal_SegmentsCoords = zeros(2 * size(Optimal_SegmentsCoords, 1) + 1, 2)
    # Calculamos copy_optimal_SegmentsCoords 
    for j in 1:2 * size(Optimal_SegmentsCoords, 1) + 1
        if j==1
            copy_Optimal_SegmentsCoords[j,:]=(Source+Optimal_SegmentsCoords[1,:])/2
        elseif j==2*size(Optimal_SegmentsCoords,1)+1
            copy_Optimal_SegmentsCoords[j,:]=(Optimal_SegmentsCoords[end,:]+Receiver)/2
        else
            if iseven(j)
                copy_Optimal_SegmentsCoords[j,:]=Optimal_SegmentsCoords[Int(j/2),:]
            else
                copy_Optimal_SegmentsCoords[j,:]=(Optimal_SegmentsCoords[Int((j-1)/2),:]+Optimal_SegmentsCoords[Int((j+1)/2),:])/2
            end
        end
    end
    #Aquí iteramos la condición inicial
    initial_SegmentsCoords = copy(copy_Optimal_SegmentsCoords)
        
    q_x,q_z,theta_i=Slowness_Spherical_2D(Source,Optimal_SegmentsCoords[1,:], c_air)
    println("Ángulo ", 2^i, "segmentos: ", theta_i)
    u0 = [q_x, q_z]
    prob = ODEProblem(q!, u0, zspan, p)
    sol = solve(prob, RK4(),dt=1e-5,saveat=Source[2]:1:500.)

    #Guardamos los valores de vec(q)
    qx_vals=zeros(length(sol.u))
    qz_vals=zeros(length(sol.u))
    for i in 1:length(sol.u)
        qx_vals[i]=sol.u[i][1]
        qz_vals[i]=sol.u[i][2]
    end
    ###### 
    qx=sum(qx_vals)/length(qx_vals)
    ######
    z_values_linear = collect(range(Source[2], stop=500., length=length(qz_vals)))
    qz_z_linear=LinearInterpolation(z_values_linear, qz_vals)
end
traveltime=Hamiltonian_l(qx, qz_z_linear, Plot_Coords[n])
println("Num segments: ", 2^n, ". Traveltime:", traveltime, " s")
P3_ImPropia=deepcopy(Plot_Coords[n]);
""
end

Ángulo 1 segmento: 78.71569404699797
Num segments: 1. Traveltime:0.22360184982950382 s
Iteraciones realizadas: 1. Evaluaciones de la función: 9
Ángulo 2segmentos: 78.64536473475624
Num segments: 2. Traveltime:0.22360038856584113 s
Iteraciones realizadas: 1. Evaluaciones de la función: 20
Ángulo 4segmentos: 78.60841411960985
Num segments: 4. Traveltime:0.22359964228652898 s
Iteraciones realizadas: 1. Evaluaciones de la función: 38
Ángulo 8segmentos: 78.5894154560883
Num segments: 8. Traveltime:0.22359926381290768 s
Iteraciones realizadas: 1. Evaluaciones de la función: 54
Ángulo 16segmentos: 78.57979353629214
Num segments: 16. Traveltime:0.2235990735715704 s
Iteraciones realizadas: 1. Evaluaciones de la función: 86
Ángulo 32segmentos: 78.57495005231291
Num segments: 32. Traveltime:0.22359897805931184 s
Iteraciones realizadas: 1. Evaluaciones de la función: 62
Ángulo 64segmentos: 78.57228148823368
Num segments: 64. Traveltime:0.22359892552134208 s
Iteraciones realizadas: 1. Evaluaciones 

""

In [17]:
using LinearAlgebra
function Analytical_RP(theta,alpha_c,c0,z)
    psi=cosd(theta)/c0 #Coseno para ángulos en grados
    x=(sqrt(1-(psi*c0)^2)-sqrt(1-psi^2*(c0+alpha_c*(z-SourcePoint[2]))^2))/(psi*alpha_c)
    return x
end
SourcePoint=[0.,100.]
alpha_c=0.6 ;
c0=332+0.6*100;
using Roots
@time begin
theta1=75
x1_coords=0.:1:60.
z1_coords=[]
for i in 1:length(x1_coords)
    function objective_function(z)
        x_target = x1_coords[i]  # reemplaza esto con x dado
        x_function = Analytical_RP(theta1, alpha_c, c0, z)
        return x_function - x_target
    end
    z_initial_guess = SourcePoint[2]# Suposición inicial para z
    z_solution = fzero(objective_function, z_initial_guess)
    push!(z1_coords,z_solution)
end
#####################
theta2=45
x2_coords=0.:1:150.
z2_coords=[]
for i in 1:length(x2_coords)
    function objective_function(z)
        x_target = x2_coords[i]  # reemplaza esto con x dado
        x_function = Analytical_RP(theta2, alpha_c, c0, z)
        return x_function - x_target
    end
    z_initial_guess = SourcePoint[2]# Suposición inicial para z
    z_solution = fzero(objective_function, z_initial_guess)
    push!(z2_coords,z_solution)
end
###########################
theta3=15
x3_coords=0.:1:86.
z3_coords=[]
for i in 1:length(x3_coords)
    function objective_function(z)
        x_target = x3_coords[i]  # reemplaza esto con x dado
        x_function = Analytical_RP(theta3, alpha_c, c0, z)
        return x_function - x_target
    end
    z_initial_guess = SourcePoint[2]# Suposición inicial para z
    z_solution = fzero(objective_function, z_initial_guess)
    push!(z3_coords,z_solution)
end
end

  0.080826 seconds (147.27 k allocations: 3.848 MiB, 77.74% compilation time)


In [16]:
function Traveltime_Noord(SourcePoint,ReceiverPoint,alpha_c,c0)
    d=ReceiverPoint[1]-SourcePoint[1]
    Delta_z=ReceiverPoint[2]-SourcePoint[2]
    xi=alpha_c/c0
    tan_Psi=xi*d/2 + Delta_z*(2+xi*Delta_z)/(2*d)
    Psi=rad2deg(atan(tan_Psi))
    function f(z)
        fz=(1+sqrt(1-(1+xi*z)^2*cosd(Psi)^2))/(1-sqrt(1-(1+xi*z)^2*cosd(Psi)^2))
        return fz
    end
    f_0=f(0)
    f_Z=f(Delta_z)
    fZ=(1+sqrt(1-(1+xi*Delta_z)^2*cosd(Psi)^2))/(1-sqrt(1-(1+xi*Delta_z)^2*cosd(Psi)^2))
    f0=(1+sind(Psi))/(1-sind(Psi))
    tau_z=log(f0/fZ)/(2*xi*c0)
    #tau_z=log(f_0/f_Z)/(2*xi*c0)
    dm=tan_Psi/xi 
    if xi>0 && d<=dm
        traveltime=tau_z
    else
        Delta_zm=(1/cosd(Psi)-1)/xi
        f_ZM=f(Delta_zm)
        tau_zm=log(f_0/f_ZM)/(2*xi*c0)

        traveltime=2*tau_zm-tau_z
    end
    return traveltime
end
alpha_c=0.6
c0=332+100*0.6
traveltime1=Traveltime_Noord([0.,100.],[x1_coords[end],z1_coords[end]],alpha_c,c0)
traveltime2=Traveltime_Noord([0.,100.],[x2_coords[end],z2_coords[end]],alpha_c,c0)
traveltime3=Traveltime_Noord([0.,100.],[x3_coords[end],z3_coords[end]],alpha_c,c0)
println(traveltime1)
println(traveltime2)
println(traveltime3)

LoadError: BoundsError: attempt to access 0-element Vector{Any} at index [0]

In [52]:
using Plots
p=plot(legend=:topright, legendfontsize=6)

# Implementación Propia
plot!(P1_ImPropia[:,1],P1_ImPropia[:,2],label="Implementation \$\\theta_i\$=72.71º, \$\\theta_f\$=74.82º : \$\\tau\$ =0.4556 s ", color=:blue)
plot!(P2_ImPropia[:,1],P2_ImPropia[:,2],label="Implementation \$\\theta_i\$=39.01º, \$\\theta_f\$=42.59º : \$\\tau\$ =0.4743 s ", color=:green)
plot!(P3_ImPropia[:,1],P3_ImPropia[:,2],label="Implementation \$\\theta_i\$=12.28º, \$\\theta_f\$=12.43º : \$\\tau\$ =0.2236 s ", color=:red)
#Analytical
plot!(x1_coords,z1_coords, label="Analytical Ray Path \$\\theta\$=75º : \$\\tau\$ =0.4511 s ", color=:blue, style=:dash)
plot!(x2_coords,z2_coords, label="Analytical Ray Path \$\\theta\$=45º : \$\\tau\$ =0.4507 s ", color=:green, style=:dash)
plot!(x3_coords,z3_coords, label="Analytical Ray Path \$\\theta\$=15º : \$\\tau\$ =0.2207 s ", color=:red, style=:dash)


# Add labels to the axes
xlabel!("x (m)")
ylabel!("z (m)")

# Set the title
title!("McBride Validation")

# Save the figure
savefig(p, "mcbride(derivadaespacio)_iterandoangulo.png")

"C:\\Users\\Miguel\\Desktop\\TFM\\Codigos Julia\\mcbride(derivadaespacio)_iterandoangulo.png"